API connection 

In [1]:
import os
from getpass import getpass

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY") or getpass("Enter API key:")


In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai/gpt-oss-20b", model_provider="groq", temperature=0.0)
creative_llm = init_chat_model("openai/gpt-oss-20b", model_provider="groq", temperature=0.9)


In [11]:
import os
from dotenv import load_dotenv
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.tools import tool

# Load environment variables
load_dotenv()
serper_api_key = os.getenv("SERPER_API_KEY")

# Initialize search tool
search = GoogleSerperAPIWrapper(serper_api_key=serper_api_key)


@tool
def google_search(query: str) -> str:
    """
    Retrievs information through google search
    """
    result = search.run(query)
    return result


In [12]:
from langchain_openai import ChatOpenAI


def set_llm(api_key=None, api_key_base=None, model="gpt-3.5-turbo"):
    """
    Initialize LLM and store it in st.session_state.llm_instance.
    Returns True if successful.
    """
    api_key
    if not api_key:
        raise ValueError("⚠️ API key not provided!")

    # Initialize LLM and store in session
    return ChatOpenAI(
        model_name=model,
        temperature=0,
        openai_api_key=api_key,
        openai_api_base=api_key_base,
    )


In [ ]:
# disease_info_tool.py
from langchain.tools import tool
from langchain_community.vectorstores import FAISS



embeddings = get_embeddings()  # or HuggingFaceEmbeddings() if that’s your fn
df = pd.read_csv(MAYO_CSV)
vectorstore = FAISS.load_local(
    DISEASE_INFO_FAISS_DB, embeddings, allow_dangerous_deserialization=True
)


@tool
def match_disease_info(query: str) -> str:
    """
    Retrieve top-3 matching diseases from the FAISS DB
    and return a short overview for each.
    """
    results = vectorstore.similarity_search(query, k=3)

    if not results:
        return "No close matches found."

    out = []
    for r in results:
        disease = r.metadata.get("disease", "Unknown")
        # get Overview from the CSV
        overview_series = df.loc[df["disease"] == disease, "Overview"]
        symptoms_series = df.loc[df["disease"] == disease, "Symptoms"]

        overview = (
            overview_series.values[0]
            if not overview_series.empty
            else "Overview not found."
        )
        symptoms = (
            symptoms_series.values[0]
            if not symptoms_series.empty
            else "Symptom not found."
        )

        out.append(f"Disease: {disease}\nOverview: {overview}\nSymptoms: {symptoms}")

    return "\n\n".join(out)


In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.tools import tool

# Load environment variables
load_dotenv()
serper_api_key = os.getenv("SERPER_API_KEY")

# Initialize search tool
search = GoogleSerperAPIWrapper(serper_api_key=serper_api_key)


@tool
def google_search(query: str) -> str:
    """
    Retrievs information through google search
    """
    result = search.run(query)
    return result


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.base import RunnableSerializable


class DISEASEINFOAGENT:
    def __init__(self, llm):
        self.system_prompt = """
        You're a helpful assistant. When you get a query, you should be able to extract
        disease info using any of the tools given.
        """
        self.tools = [match_disease_info, google_search]
        self.llm = llm

        # create the prompt template
        self.prompt_template = ChatPromptTemplate.from_messages(
            [("system", self.system_prompt), ("human", "{query}")]
        )

        # create the agent once
        self.agent: RunnableSerializable = (
            {"query": lambda x: x["query"]}
            | self.prompt_template
            | self.llm.bind_tools(self.tools, tool_choice="any")
        )

    def invoke(self, query):
        response_str = self.agent.invoke({"query": query})
        return response_str


In [16]:
load_dotenv()

llm = set_llm(os.getenv("GROQ_API_KEY"), os.getenv("TEST_API_BASE"), os.getenv("TEST_MODEL") )
d =DECIDERAGENT(llm)

print(d.invoke("what is common cold"))

content='disease_info' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 174, 'total_tokens': 327, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.052476069, 'prompt_time': 0.008725021, 'completion_time': 0.303980181, 'total_time': 0.312705202}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_3a688838c3', 'id': 'chatcmpl-f8731278-7f46-4732-b37b-984b84e3282c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--3d27f1bf-006a-4997-95f4-9d7728cddd8d-0' usage_metadata={'input_tokens': 174, 'output_tokens': 153, 'total_tokens': 327, 'input_token_details': {}, 'output_token_details': {}}


In [5]:
creative_llm.invoke("Hello WOrld")

AIMessage(content='Hello! 🌍 How can I help you today?', additional_kwargs={'reasoning_content': 'The user says "Hello WOrld". It\'s likely a greeting. Should respond politely. Possibly mention the assistant name? The user says "Hello WOrld". The correct spelling would be "Hello World". Could respond with greeting and ask how can help. Should I correct the spelling? The user typed "WOrld" with capital O in middle. It\'s okay to correct politely. Let\'s greet.'}, response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 75, 'total_tokens': 177, 'completion_time': 0.09254194, 'prompt_time': 0.004819386, 'queue_time': 0.046388904, 'total_time': 0.097361326}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_a5ac2a5d7b', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a241ed8f-7a63-40f6-9c61-f9560d37686a-0', usage_metadata={'input_tokens': 75, 'output_tokens': 102, 'total_tokens': 177})

In [35]:
article = """
If there is no enemy within, the enemy outside can do us no harm.”1

Ever since a Hezbollah suicide bomber in 1983 blew up a truck packed with explosives and killed 241 Marines in Beirut, combating Islamic terrorist organizations has been a priority for U.S. intelligence, security, and law enforcement agencies. However, for those of us who have followed the spiraling growth of Islamic terrorism in the 1980s and 1990s, it seemed as if the U.S was sluggishly reactive. They made little headway in extensive counterterrorism programs designed at penetrating and dismantling Islamic terror groups.2

How was it possible for the hijackers and their plot to remain off the radar of intelligence and law enforcement?
The 9/11 attack put a spotlight on the failures of the security agencies tasked to protect the U.S. against acts of terror. How was it possible for 19 hijackers and their ambitious plot to remain off the radar of intelligence and law enforcement? The truth, as I discovered during 18 months of reporting for my book, Why America Slept: The Failure to Prevent 9/11, was not that the plot had gone undetected, but rather that the agencies responsible for monitoring and fighting terrorism had failed to share information, something that would have made it possible to connect the dots before the attack occurred.

The failures were more substantive than mere interagency rivalries between the CIA, FBI, NSA, and local law enforcement. Exclusive interviews I had with top intelligence officers and FBI officials revealed that the origins and depth of the dysfunction inside America’s counterintelligence programs was an internecine bureaucratic war that left little room for working together. Sharing information was given lip service but seldom practiced, particularly when the intelligence at stake was judged as having “high value.”

If the CIA had alerted the State Department, the two Saudis would have been on a watch list that barred them from entering the United States."""

In [36]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant and your name is {name} that helps generate article titles",
    input_variables=["name"]
)

user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a name for a article. The article is here for you to examine:

    ---

    {article}

    ---

    The name should be based of the conext of the article. Be creative, but make sure the names are clear, catchy,
    and relavant to the theme of the article.

    Only output the article name, no other explaination or
    text   can be provided. """,
    input_variables=["article"]
)

In [37]:
user_prompt.format(name="joe",article="TEST STRING").content

'You are tasked with creating a name for a article. The article is here for you to examine:\n\n    ---\n\n    TEST STRING\n\n    ---\n\n    The name should be based of the conext of the article. Be creative, but make sure the names are clear, catchy,\n    and relavant to the theme of the article.\n\n    Only output the article name, no other explaination or\n    text   can be provided. '

In [38]:
from langchain.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [39]:
first_prompt.format(name="joe",article="TEST_STRING")

'System: You are an AI assistant and your name is joe that helps generate article titles\nHuman: You are tasked with creating a name for a article. The article is here for you to examine:\n\n    ---\n\n    TEST_STRING\n\n    ---\n\n    The name should be based of the conext of the article. Be creative, but make sure the names are clear, catchy,\n    and relavant to the theme of the article.\n\n    Only output the article name, no other explaination or\n    text   can be provided. '

In [ ]:
chain_one = (
    {"article": lambda x: x["article"],
     "name": lambda x:x["name"]}
    | first_prompt
    | creative_llm
    | {"article_title": lambda x: x.content}
)

In [41]:
article_title_msg= chain_one.invoke({
    "article": article,
    "name": "Joe"
})

In [42]:
article_title_msg

{'article_title': 'The Silent War Within: How U.S. Intelligence Silos Allowed 9/11'}

In [43]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps build good articles"
)

second_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a descripion for the
      article. The article is here for you to examine:

    ---

    {article}

    ---

    Here is the article title: {article_title}

    Ouput SEO friendly article description. Do not output anything other than description.

""",
    input_variables=["article", "article_title"]
)

In [44]:
second_prompt= ChatPromptTemplate([system_prompt, second_user_prompt])

In [45]:
chain_two = (
    {"article": lambda x:x["article"],
     "article_title": lambda x:x["article_title"]}
    | second_prompt
    | llm
    | {"Summary": lambda x:x.content}
)


In [46]:
description_msg = chain_two.invoke(
    {"article" : article,
     "article_title" :      article_title_msg["article_title"]}
)

In [47]:
description_msg

{'Summary': 'Explore how U.S. intelligence silos and interagency rivalry let 9/11 terrorists slip through the cracks, revealing a failure of information sharing and counterterrorism.'}

In [76]:
paragraph = """
Hi, I am Harmesh. I am currently pursuing my B.Tech in Computer Science at Lovely Professional University. I specialize in Data Science and Machine Learning."""

In [ ]:
third_system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps to review and rewrite given paragraph"
)

third_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with reviewing the paragraph
       and rewrite it with proper grammar and wrting. The paragraph is here for you to examine:

    ---

    {paragraph}

    ---

    Choose one paragraph to review and edit. During your edit ensure you provide
    constructive feedback to the user so they can learn where to improve their own writing.,


""",
    input_variables=["paragraph"]
)

In [78]:
third_prompt = ChatPromptTemplate([third_system_prompt, third_user_prompt])

In [79]:
from pydantic import BaseModel, Field

class Paragraph(BaseModel):
    original_paragraph: str = Field(descripion = "The original Paragraph"),
    edited_paragraph: str = Field(description="The improved edited Paragraph"),
    feedback: int = Field(description="Constructive feedback about the original paragrpah out of 100")

structured_llm = creative_llm.with_structured_output(Paragraph)

/home/ml/Documents/RAG-Experiments/.venv/lib/python3.12/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'descripion': 'The original Paragraph'}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/home/ml/Documents/RAG-Experiments/.venv/lib/python3.12/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The improved edited Paragraph'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [85]:
chain_three= (
    {"paragraph": lambda x:x["paragraph"]}
    | third_prompt
    | structured_llm
    | {"output": lambda x:x.edited_paragraph}
)

In [86]:
structured_msg = chain_three.invoke({"paragraph":paragraph})

In [87]:
structured_msg

{'output': 'Hello, I’m Harmesh. I am currently pursuing a B.Tech in Computer Science at Lovely Professional University, where I specialize in Data Science and Machine Learning.'}

In [ ]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.prompts import PromptTemplate

image_prompt = PromptTemplate(
    input_variables=["article"],
    template="""
    Generate a prompt with less than 500 characters to generate an image
    based on the following article ; {article}"""
)

## LangChain's Memory Types

LangChain versions `0.0.x` consisted of various conversational memory types. Most of these are due for deprecation but still hold value in understanding the different approaches that we can take to building conversational memory.

Throughout the notebook we will be referring to these _older_ memory types and then rewriting them using the recommended `RunnableWithMessageHistory` class. We will learn about:

* `ConversationBufferMemory`: the simplest and most intuitive form of conversational memory, keeping track of a conversation without any additional bells and whistles.
* `ConversationBufferWindowMemory`: similar to `ConversationBufferMemory`, but only keeps track of the last `k` messages.
* `ConversationSummaryMemory`: rather than keeping track of the entire conversation, this memory type keeps track of a summary of the conversation.
* `ConversationSummaryBufferMemory`: merges the `ConversationSummaryMemory` and `ConversationTokenBufferMemory` types.

We'll work through each of these memory types in turn, and rewrite each one using the `RunnableWithMessageHistory` class.

## 1. `ConversationBufferMemory`

`ConversationBufferMemory` is the simplest form of conversational memory, it is _literally_ just a place that we store messages, and then use to feed messages into our LLM.

Let's start with LangChain's original `ConversationBufferMemory` object, we are setting `return_messages=True` to return the messages as a list of `ChatMessage` objects — unless using a non-chat model we would always set this to `True` as without it the messages are passed as a direct string which can lead to unexpected behavior from chat LLMs.

In [6]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

/tmp/ipykernel_108827/1448044083.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [7]:
memory.save_context(
    {"input": "Hi, my name is James"},  # user message
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}  # AI response
)
memory.save_context(
    {"input": "I'm researching the different types of conversational memory."},  # user message
    {"output": "That's interesting, what are some examples?"}  # AI response
)
memory.save_context(
    {"input": "I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory."},  # user message
    {"output": "That's interesting, what's the difference?"}  # AI response
)
memory.save_context(
    {"input": "Buffer memory just stores the entire conversation, right?"},  # user message
    {"output": "That makes sense, what about ConversationBufferWindowMemory?"}  # AI response
)
memory.save_context(
    {"input": "Buffer window memory stores the last k messages, dropping the rest."},  # user message
    {"output": "Very cool!"}  # AI response
)

In [8]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMess

In [9]:
memory = ConversationBufferMemory(return_messages=True)

memory.chat_memory.add_user_message("Hi, my name is James")
memory.chat_memory.add_ai_message("Hey James, what's up? I'm an AI model called Zeta.")
memory.chat_memory.add_user_message("I'm researching the different types of conversational memory.")
memory.chat_memory.add_ai_message("That's interesting, what are some examples?")
memory.chat_memory.add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("That's interesting, what's the difference?")
memory.chat_memory.add_user_message("Buffer memory just stores the entire conversation, right?")
memory.chat_memory.add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
memory.chat_memory.add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
memory.chat_memory.add_ai_message("Very cool!")

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMess

In [10]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

/tmp/ipykernel_108827/839683240.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(


In [11]:
chain.invoke({"input": "what is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's the difference?", additional_kw

{'input': 'what is my name again?',
 'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}

### `ConversationBufferMemory` with `RunnableWithMessageHistory`

As mentioned, the `ConversationBufferMemory` type is due for deprecation. Instead, we can use the `RunnableWithMessageHistory` class to implement the same functionality.

When implementing `RunnableWithMessageHistory` we will use **L**ang**C**hain **E**xpression **L**anguage (LCEL) and for this we need to define our prompt template and LLM components. Our `llm` has already been defined, so now we just define a `ChatPromptTemplate` object.

In [13]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Zeta."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])

In [14]:
pipeline = prompt_template | llm

In [15]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]